# Define Parameters

In [157]:
import time 
import math
start_time = time.time()

In [158]:
import gurobipy

In [159]:
# Which linear progamming API am I using?
# Do not currently have a Gurobi license so puLP it is!  
gurobi_lp = False 

# Variables for event 
number_sessions =  1
max_number_groups = 20 
max_group_size = 8
number_groups_dict = {1:20 , 2: 20, 3: 20, 4: 20, 5: 20, 6: 20, 7: 20, 8: 20}

main_sessions = [i for i in range(1, number_sessions+1)]


# When this is true the scheduling conflicts are included, 
# if changed to false they are removed and everyone is included 
scheduling_conflicts_on = True


# Reporting Outputs 
write_to_DB = False 
write_to_excel = True

cancelled_contacts = []

## Import Packages 

In [160]:
# Numerical and Data Manipulation
import numpy as np 
import pandas as pd 
import math
import statistics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Functional Programming
from functools import reduce 
import json 
import pickle 

# Copying and Randomization
import copy
import random

# Iteration and Combinations
import itertools
from itertools import repeat, combinations, permutations, combinations_with_replacement

# Collections
from collections import defaultdict, Counter

# Excel and Data Handling
import xlsxwriter
from openpyxl.utils import get_column_letter

# Database Connectivity
# import pyodbc
# import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, table, Column, Integer, String

# Text Manipulation
import re

# Date and Time
from datetime import datetime, timedelta

# Progress Bar
from tqdm import tqdm

# String Matching
from fuzzywuzzy import fuzz

# Word Document Manipulation
# from docx import Document
# from docx.shared import Cm

# Graph Theory
import networkx as nx

# Linear Programming
import pulp
from pulp import lpSum, LpMaximize
import gurobipy as grb
from gurobipy import Model, GRB

# System and Process Information
import psutil
import sys

# Excel and Data Handling
import xlsxwriter

# Set random seed for reproducibility
np.random.seed(24)

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

import string

## Define Functions

In [161]:
# Sorts the (A) and (F) in the dataframe
def sort_group_by_marker(group, session_columns):
    # Extract the last 3 characters from each session column
    group[session_columns] = group[session_columns].apply(lambda x: x.str[-3:])
    
    # Create a list of session columns to use for sorting
    sort_columns = list(session_columns)
    
    # Sort the DataFrame by the specified session columns
    group = group.sort_values(by=sort_columns, ascending=True)
    
    # Drop the temporary columns used for sorting
    group = group.drop(session_columns, axis=1)
    
    return group

# Print red bold text 
def print_in_red_and_large(text):
    formatted_text = f"\033[91m\033[1m{text}\033[0m"
    print(formatted_text)


# Write a table to the database
def create_database_table(table_name, df_name, column_names, engine):
    # Create the metadata object
    metadata = MetaData()

    # Define the table
    table = Table(
        table_name,
        metadata,
        Column('ID', Integer, primary_key=True),  # Add an ID column as primary key
        *(Column(col, String(255), nullable=False) for col in column_names)
    )

    # Create the table in the database
    metadata.create_all(engine)

    # Ensure the group is created
    groups = pd.read_sql("select * from INFORMATION_SCHEMA.TABLES", engine)
    if table_name in tables['TABLE_NAME'].values:
        print(f"Table '{table_name}' created successfully.")
    else:
        print(f"Error: Table '{table_name}' not found in the database.")

# Changes a name from first_last_A to First Last (A)
def convert_name_format(name):
    parts = name.split('_')
    first_name = parts[0].capitalize()
    last_name = parts[1].capitalize()
    group = parts[2].upper()
    formatted_name = f"{first_name} {last_name}  ({group})"
    return formatted_name

# Define a function to extract the session number
def extract_session_number(title):
    match = re.search(r'#(\d+)', title)
    if match:
        session_number = match.group(1)
        return f"Session {session_number}"
    else:
        return None

# Import Data
- Data was randomly generated by ChatGPT using the prompt: In table format, give me 120 first and last names (where these are the two columns). first names and last names can be repeated, but no first-last name combination can be repeated. Random words as last names are fine. 

In [162]:
df_db = pd.read_excel(r"attendee_data.xlsx", sheet_name= 'all reg')
# df_db = pd.read_excel(r"/Users/summerpurschke/Desktop/attendee_data.xlsx", sheet_name= 'all reg')

strip whitespace and add CompanyType marker 
- company types are A or B

In [163]:
## Queries 
df_db['FirstName'] = df_db['FirstName'].str.strip()
df_db['LastName'] = df_db['LastName'].str.strip()

# Add contact to database to compare
condition = df_db['CompanyType'] == 'A'
df_db.loc[condition, 'Contact'] = df_db.loc[condition, 'FirstName'].str.lower() + '_' + df_db.loc[condition, 'LastName'].str.lower() + "_A"

condition = df_db['CompanyType'] == 'B'
df_db.loc[condition, 'Contact'] = df_db.loc[condition, 'FirstName'].str.lower() + '_' + df_db.loc[condition, 'LastName'].str.lower() + "_B"

#####  remove cancelled people 

In [164]:
df_db = df_db[~df_db['Contact'].isin(cancelled_contacts)]

##### Scheduling Conflicts

Create mock data where every ContactID shows every session

In [165]:
# Define the range of ContactIDs and the Titles
contact_ids = range(1, 120)
titles = [f'Session #{i}' for i in range(1, 9)]

# Create a list of dictionaries with the desired structure
data = []
for contact_id in contact_ids:
    for title in titles:
        data.append({'Title': title, 'ContactId': contact_id})

# Create the DataFrame
scheduling_conflicts_df_db = pd.DataFrame(data)

# scheduling_conflicts_df_db

# Preprocess

In [166]:
# Split into A and B df
groupB_df_db  =  df_db[df_db['CompanyType'] == 'B']
groupA_df_db  =  df_db[df_db['CompanyType'] == 'A']

In [167]:
# Create a contact field 
groupA_df_db['Contact'] = groupA_df_db['FirstName'].str.lower() + '_' + groupA_df_db['LastName'].str.lower() + '_A'
groupB_df_db['Contact'] = groupB_df_db['FirstName'].str.lower() + '_' + groupB_df_db['LastName'].str.lower() + '_B'

# Fold first and last name to lower 
groupA_df_db['FirstName'] = groupA_df_db['FirstName'].str.lower()
groupA_df_db['LastName'] = groupA_df_db['LastName'].str.lower()

groupB_df_db['FirstName'] = groupB_df_db['FirstName'].str.lower()
groupB_df_db['LastName'] = groupB_df_db['LastName'].str.lower()

# Rename to match what I have been using in the following code
groupB_df_db = groupB_df_db.rename(columns = {'Asset Class 1': 'Asset Class Theme Pref 1','Asset Class 2': 'Asset Class Theme Pref 2' })

In [168]:
groupB_df_db['Preference'] = groupB_df_db['Preference'].fillna('')
groupB_df_db['NoPreference'] = groupB_df_db['NoPreference'].fillna('')

##### Preprocess preference and no preference names

In [169]:
# Replace whitespace values with None (null)
groupB_df_db.loc[groupB_df_db['Preference'].str.strip() == '', 'Preference'] = None

if groupB_df_db['Contact'].nunique() == groupB_df_db.shape[0]:
    for contact in groupB_df_db['Contact'].unique():
        contact_rows = groupB_df_db['Contact'] == contact
        if groupB_df_db.loc[contact_rows, 'Preference'].iloc[0] is not None:
            # Get the preference values for the current contact
            names_string = groupB_df_db.loc[contact_rows, 'Preference'].iloc[0]
            names_list = names_string.split(',')

            # Strip whitespace, fold to lowercase, and replace spaces with underscores
            cleaned_names_list = [name.strip().lower().replace(' ', '_') + '_A' for name in names_list]

            # Update the "Preference" column with the formatted names
            groupB_df_db.loc[contact_rows, 'Preference'] = ','.join(cleaned_names_list)

In [170]:
# Replace whitespace values with None (null)
groupB_df_db.loc[groupB_df_db['NoPreference'].str.strip() == '', 'NoPreference'] = None

if groupB_df_db['Contact'].nunique() == groupB_df_db.shape[0]:
    for contact in groupB_df_db['Contact'].unique():
        contact_rows = groupB_df_db['Contact'] == contact
        if groupB_df_db.loc[contact_rows, 'NoPreference'].iloc[0] is not None:
            # Get the preference values for the current contact
            names_string = groupB_df_db.loc[contact_rows, 'NoPreference'].iloc[0]
            names_list = names_string.split(',')

            # Strip whitespace, fold to lowercase, and replace spaces with underscores
            cleaned_names_list = [name.strip().lower().replace(' ', '_') + '_A' for name in names_list]

            # Update the "Preference" column with the formatted names
            groupB_df_db.loc[contact_rows, 'NoPreference'] = ','.join(cleaned_names_list)


In [171]:
groupA_df_db['Preference'] = groupA_df_db['Preference'].fillna('')

# Replace whitespace values with None (null)
groupA_df_db.loc[groupA_df_db['Preference'].str.strip() == '', 'Preference'] = None

if groupA_df_db['Contact'].nunique() == groupA_df_db.shape[0]:
    # print('ok')
    for contact in groupA_df_db['Contact'].unique():
        contact_rows = groupA_df_db['Contact'] == contact
        if groupA_df_db.loc[contact_rows, 'Preference'].iloc[0] is not None:
            # Get the preference values for the current contact
            names_string = groupA_df_db.loc[contact_rows, 'Preference'].iloc[0]
            names_list = names_string.split(',')

            # Strip whitespace, fold to lowercase, and replace spaces with underscores
            cleaned_names_list = [name.strip().lower().replace(' ', '_') + '_B' for name in names_list]

            # Update the "Preference" column with the formatted names
            groupA_df_db.loc[contact_rows, 'Preference'] = ','.join(cleaned_names_list)


In [172]:
groupB_df_db['NoPreference'] = groupB_df_db['NoPreference'].str.replace('__', '_')
groupB_df_db['Preference'] = groupB_df_db['Preference'].str.replace('__', '_')

groupA_df_db['NoPreference'] = groupA_df_db['NoPreference'].str.replace('__', '_')
groupA_df_db['Preference'] = groupA_df_db['Preference'].str.replace('__', '_')


##### *Remove Cancelled Contacts from prefrences*

In [173]:
for name in cancelled_contacts:
    if name.endswith('_B'):
        groupB_df_db['Preference'] = groupB_df_db['Preference'].str.replace(name, '')
        groupB_df_db['Preference'] = groupB_df_db['Preference'].str.replace(',,', ',')

        groupB_df_db['NoPreference'] = groupB_df_db['NoPreference'].str.replace(name, '')
        groupB_df_db['NoPreference'] = groupB_df_db['NoPreference'].str.replace(',,', ',')
    elif name.endswith('_A'):
        groupA_df_db['Preference'] = groupA_df_db['Preference'].str.replace(name, '')
        groupA_df_db['Preference'] = groupA_df_db['Preference'].str.replace(',,', ',')

#### Scheduling Conflicts DB Data

##### Preprocessing Scheduling conflicts data

In [174]:
common_contact_ids = set(groupA_df_db['ContactID'].unique()).intersection(set(groupB_df_db['ContactID'].unique()))
if len(set(common_contact_ids)) > 1: raise ValueError

In [175]:
if scheduling_conflicts_on:
    # list of A and B
    A_and_B_contacts = list(set(groupA_df_db['ContactID'].unique()) | set(groupB_df_db['ContactID'].unique()))

    # Filter down to only contacts that are relevant
    scheduling_conflicts_df = scheduling_conflicts_df_db[scheduling_conflicts_df_db['ContactId'].isin(A_and_B_contacts)]

    # Apply the function to create the 'Session' column
    # function is in the define functions area
    scheduling_conflicts_df['Session'] = scheduling_conflicts_df['Title'].apply(extract_session_number)

    # Assign Role column based on where the contactID lives 
    scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'].isin(groupA_df_db['ContactID'].unique()), 'Role'] = 'A'
    scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'].isin(groupB_df_db['ContactID'].unique()), 'Role'] = 'B'

    scheduling_conflicts_df['FirstName'] = np.nan
    scheduling_conflicts_df['LastName'] = np.nan

    for contact_id in scheduling_conflicts_df['ContactId'].unique():

        # A
        if scheduling_conflicts_df[scheduling_conflicts_df['ContactId'] == contact_id]['Role'].unique()[0] == 'A':

            # Pull First name from A df
            firstname = groupA_df_db[groupA_df_db['ContactID'] == contact_id]['FirstName'].values[0]
            scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'] == contact_id, 'FirstName'] = firstname
            # Pull Last name from A data
            lastname = groupA_df_db[groupA_df_db['ContactID'] == contact_id]['LastName'].values[0]
            scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'] == contact_id, 'LastName'] = lastname

        # B
        elif scheduling_conflicts_df[scheduling_conflicts_df['ContactId'] == contact_id]['Role'].unique()[0] == 'B':
            # Pull First name from B data
            scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'] == contact_id, 'FirstName'] = groupB_df_db[groupB_df_db['ContactID'] == contact_id]['FirstName'].values[0]
            # Pull Last name from B data
            scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'] == contact_id, 'LastName'] = groupB_df_db[groupB_df_db['ContactID'] == contact_id]['LastName'].values[0]

    # Rearrange the columns 
    scheduling_conflicts_df = scheduling_conflicts_df[['FirstName', 'LastName', 'Role', 'Session', 'Title', 'ContactId']]


## Define Dictionaries 

In [176]:
groupB = list(groupB_df_db['Contact'].unique())
groupA = list(groupA_df_db['Contact'].unique())
attendees = list(set(groupB).union(set(groupA)))

Create Mock Data for Preferences
- This stays commented out, only used to create or change the mock data 

In [177]:
# df_db['Name'] = df_db['FirstName'].str.strip() + ' ' + df_db['LastName'].str.strip() 
# groupA_names = list(df_db[df_db['CompanyType'] == 'A']['Name'].unique())
# groupB_names = list(df_db[df_db['CompanyType'] == 'B']['Name'].unique())

# # Function to assign preferences skewed towards 5 values
# def assign_preferences(names_list):
#     # Weighted distribution of number of names to choose
#     num_choices = random.choices([0, 1, 2, 3, 4, 5], weights=[1, 1, 1, 3, 4, 10])[0]
#     if num_choices == 0:
#         return ''
#     return ', '.join(random.sample(names_list, num_choices))

# # Apply function to rows where CompanyType is 'A'
# df_db.loc[df_db['CompanyType'] == 'A', 'Preference'] = df_db.loc[df_db['CompanyType'] == 'A'].apply(
#     lambda row: assign_preferences(groupB_names), axis=1)

# # Apply function to rows where CompanyType is 'B'
# df_db.loc[df_db['CompanyType'] == 'B', 'Preference'] = df_db.loc[df_db['CompanyType'] == 'B'].apply(
#     lambda row: assign_preferences(groupA_names), axis=1)



# # Function to assign preferences skewed towards 5 values
# def assign_nopreferences(names_list):
#     # Weighted distribution of number of names to choose
#     num_choices = random.choices([0, 1, 2, 3], weights=[10,10,5,1])[0]
#     if num_choices == 0:
#         return ''
#     return ', '.join(random.sample(names_list, num_choices))

# # Apply function to rows where CompanyType is 'B'
# df_db.loc[df_db['CompanyType'] == 'B', 'NoPreference'] = df_db.loc[df_db['CompanyType'] == 'B'].apply(
#     lambda row: assign_nopreferences(groupB_names), axis=1)


# df_db.to_excel(r"/Users/summerpurschke/Desktop/attendee_data2.xlsx")

##### **Preferences**

In [178]:
# Remove cancelled people from prefs
for contact in cancelled_contacts:
    groupA_df_db['Preference'] = groupA_df_db['Preference'].str.replace(contact, '')
    groupB_df_db['Preference'] = groupB_df_db['Preference'].str.replace(contact, '')
    groupB_df_db['NoPreference'] = groupB_df_db['NoPreference'].str.replace(contact, '')

In [179]:
# Create a dictionary with 'Contact' as keys and 'Preference' as values
groupA_prefs_dict = {}

for contact, preference in zip(groupA_df_db['Contact'], groupA_df_db['Preference']):
    if preference is not None:
        groupA_prefs_dict[contact] = preference.split(',')
    if preference is None:
        groupA_prefs_dict[contact] = None

if set(list(groupA_prefs_dict.keys())) != set(groupA):
    raise ValueError('Discrepency with members in pref dict')

In [180]:
groupA_prefs_dict = {key: [value for value in values if value != ''] if values is not None else None for key, values in groupA_prefs_dict.items()}

In [181]:
# Create a dictionary with 'Contact' as keys and 'Preference' as values
groupB_prefs_dict = {}

for contact, preference in zip(groupB_df_db['Contact'], groupB_df_db['Preference']):
    if preference is not None:
        groupB_prefs_dict[contact] = preference.split(',')
    if preference is None:
        groupB_prefs_dict[contact] = None

if set(list(groupB_prefs_dict.keys())) != set(groupB):
    raise ValueError('Discrepency with members in pref dict')

In [182]:
groupB_prefs_dict = {key: [value for value in values if value != ''] if values is not None else None for key, values in groupB_prefs_dict.items()}

In [183]:
# Create a dictionary with 'Contact' as keys and 'Preference' as values
groupB_noprefs_dict = {}

for contact, preference in zip(groupB_df_db['Contact'], groupB_df_db['NoPreference']):
    if preference is not None:
        groupB_noprefs_dict[contact] = preference.split(',')
    if preference is None:
        groupB_noprefs_dict[contact] = None

if set(list(groupB_noprefs_dict.keys())) != set(groupB):
    raise ValueError('Discrepency with members in pref dict')

In [184]:
groupB_prefs_dict = {key: [value for value in values if value != ''] if values is not None else None for key, values in groupB_noprefs_dict.items()}

##### Create lists of preference pairs
##### - Then sort the tuples to ensure consistency

In [185]:
groupA_pref_pairs = [(key, value) for key, values in groupA_prefs_dict.items() if values is not None for value in values]
# Sort tuples into ((F,A))
temp_list = []
for pair in groupA_pref_pairs:
    personA = [person for person in pair if person.endswith('_A')][0]
    personB = [person for person in pair if person.endswith('_B')][0]
    temp_list.append((personA, personB))
groupA_pref_pairs = temp_list
# Remove tuples where the preferred personB is not in this event 
groupA_pref_pairs = [pair for pair in groupA_pref_pairs if any(item in groupB for item in pair)]

In [186]:
groupB_pref_pairs = [(key, value) for key, values in groupB_prefs_dict.items() if values is not None for value in values]
# Sort tuples into ((F,A))
temp_list = []
for pair in groupB_pref_pairs:
    personA = [person for person in pair if person.endswith('_A')][0]
    personB = [person for person in pair if person.endswith('_B')][0]
    temp_list.append((personA, personB))
groupB_pref_pairs = temp_list
# Remove tuples where the preferred personB is not in this event 
groupB_pref_pairs = [pair for pair in groupB_pref_pairs if any(item in groupA for item in pair)]

In [187]:
groupB_nopref_pairs = [(key, value) for key, values in groupB_noprefs_dict.items() if values is not None for value in values]
# Sort tuples into ((F,A))
temp_list = []
for pair in groupB_nopref_pairs:
    personA = [person for person in pair if person.endswith('_A')][0]
    personB = [person for person in pair if person.endswith('_B')][0]
    temp_list.append((personA, personB))
groupB_nopref_pairs = temp_list
# Remove tuples where the preferred personB is not in this event 
groupB_nopref_pairs = [pair for pair in groupB_nopref_pairs if any(item in groupA for item in pair)]

##### List of mutual preferences

In [188]:
mutual_pref_pairs = set(groupA_pref_pairs).intersection(set(groupB_pref_pairs))

##### **Conflicts**


In [189]:
if scheduling_conflicts_on:
    # Separate them based on F or A 
    scheduling_conflicts_A = scheduling_conflicts_df.loc[scheduling_conflicts_df['Role'] == 'A']
    scheduling_conflicts_B = scheduling_conflicts_df.loc[scheduling_conflicts_df['Role'] == 'B']

    scheduling_conflicts_A['Contact'] = scheduling_conflicts_A['FirstName'].str.lower().str.strip() + '_' + scheduling_conflicts_A['LastName'].str.lower().str.strip() + '_A'
    scheduling_conflicts_B['Contact'] = scheduling_conflicts_B['FirstName'].str.lower().str.strip() + '_' + scheduling_conflicts_B['LastName'].str.lower().str.strip() + '_B'

##### These dictionaries are a list of people that ARE Available in each session

In [190]:
if scheduling_conflicts_on:
    # Group A 
    groupA_scheduling_conflicts = {}
    for i in range(1, number_sessions+1):
        session_name = f"Session {i}"
        groupA_scheduling_conflicts [f"meeting_{i}_conflicts"] = list(set((scheduling_conflicts_A.loc[scheduling_conflicts_A['Session'] == session_name])['Contact'].to_list()))

    # if the contactID is NOT in scheduling_conflicts_A, add the name to every session 
    groupA_without_availability  = groupA_df_db[groupA_df_db['ContactID'].isin(list(set(groupA_df_db['ContactID'].unique())- set(scheduling_conflicts_A['ContactId'].unique())))]['Contact'].tolist()
    for i in range(1, number_sessions+1):
        groupA_scheduling_conflicts [f"meeting_{i}_conflicts"] = groupA_scheduling_conflicts [f"meeting_{i}_conflicts"] + groupA_without_availability

    # Group B 
    groupB_scheduling_conflicts = {}
    for i in range(1, number_sessions+1):
        session_name = f"Session {i}"
        groupB_scheduling_conflicts [f"meeting_{i}_conflicts"] = list(set((scheduling_conflicts_B.loc[scheduling_conflicts_B['Session'] == session_name])['Contact'].to_list()))

    # if the contactID is NOT in scheduling_conflicts_A, add the name to every session 
    groupB_without_availability  = groupB_df_db[groupB_df_db['ContactID'].isin(list(set(groupB_df_db['ContactID'].unique())- set(scheduling_conflicts_B['ContactId'].unique())))]['Contact'].tolist()
    for i in range(1, number_sessions+1):
        groupB_scheduling_conflicts [f"meeting_{i}_conflicts"] = groupB_scheduling_conflicts [f"meeting_{i}_conflicts"] + groupB_without_availability



else: 
    groupA_scheduling_conflicts = {}
    for i in range(1, number_sessions+1):
        groupA_scheduling_conflicts [f"meeting_{i}_conflicts"] = set(groupA)

    groupB_scheduling_conflicts = {}
    for i in range(1, number_sessions+1):
        groupB_scheduling_conflicts [f"meeting_{i}_conflicts"] = set(groupB)


In [191]:
# Check for people missing from different sessions
for i in range(1, number_sessions+1):
    print(len(groupA_scheduling_conflicts[f"meeting_{i}_conflicts"]))

42


In [192]:
# Check for people missing from different sessions
for i in range(1, number_sessions+1):
    print(len(groupB_scheduling_conflicts[f"meeting_{i}_conflicts"]))

77


##### **Dictionary of which sessions people ARE available**

In [193]:
if scheduling_conflicts_on:

    # GroupB 
    groupB_available_sessions ={}
    for contact in scheduling_conflicts_B['Contact'].unique().tolist():
        groupB_available_sessions[contact] = [int(re.search(r'\d+', session).group()) for session in scheduling_conflicts_B[scheduling_conflicts_B['Contact'] == contact]['Session']]

    # For members of groupB that did not fill out availability - assume available for all 
    for contact in groupB_without_availability:
        groupB_available_sessions[contact] = [i for i in range(1, number_sessions+1)]

    # GroupA
    groupA_available_sessions ={}
    for contact in scheduling_conflicts_A['Contact'].unique().tolist():
        groupA_available_sessions[contact] = [int(re.search(r'\d+', session).group()) for session in scheduling_conflicts_A[scheduling_conflicts_A['Contact'] == contact]['Session']]

    # For members of groupA that did not fill out availability - assume available for all 
    for contact in groupA_without_availability:
        groupA_available_sessions[contact] = [i for i in range(1, number_sessions+1)]

else:
    groupA_available_sessions = {groupA: list(range(1, number_sessions+1)) for personA in groupA}
    groupB_available_sessions = {groupB: list(range(1, number_sessions+1)) for personB in groupB}

##### **Group B No Prefs**

In [194]:
# Create a dictionary with 'Contact' as keys and 'Preference' as values
groupB_no_prefs_dict = {}

for contact, preference in zip(groupB_df_db['Contact'], groupB_df_db['NoPreference']):
    if preference is not None:
        groupB_no_prefs_dict[contact] = preference.split(',')
    if preference is None:
        groupB_no_prefs_dict[contact] = None

if set(list(groupB_no_prefs_dict.keys())) != set(groupB):
    raise ValueError('Discrepency with groupB in pref dict')

In [195]:
groupB_no_prefs_pairs = [(key, value) for key, values in groupB_no_prefs_dict.items() if values is not None for value in values]
# Sort tuples 
temp_list = []
for pair in groupB_no_prefs_pairs:
    personA = [person for person in pair if person.endswith('_A')][0]
    personB = [person for person in pair if person.endswith('_B')][0]
    temp_list.append((personA,personB))
groupB_no_prefs_pairs = temp_list
# Remove tuples where the preferred personA is not in this event 
groupB_no_prefs_pairs = [pair for pair in groupB_no_prefs_pairs if any(item in groupA for item in pair)]

##### **Pair Counts**
- generate a list of pairs and then sort the tuples for consistent
- Blow out to sessions or groups, look at combinations of groups in sessions and then go from there 
- How many combinations of A/B group structures are there - look at all of these in combination form and then aim to differ the following groups from this. 

In [196]:
# Generate combinations
pair_count_list = list(combinations(attendees, 2))
pair_count_list = list(set(pair_count_list)) # remove duplicates


# Sort each tuple within the list
temp_list = []
for pair in pair_count_list:
    # If its two people from group A or Group B
    if (pair[0] in groupA and pair[1] in groupA) or (pair[0] in groupB and pair[1] in groupB):
        temp_list.append((pair[0],pair[1]))
    # If its one person from each group 
    else:

        personA = [person for person in pair if person.endswith('_A')][0]
        personB = [person for person in pair if person.endswith('_B')][0]
        temp_list.append((personA,personB))

pair_count_list = temp_list

##### **PersonA / PersonB Pairs**

In [197]:
AB_pairs = []
for personA in groupA:
    for personB in groupB:
        AB_pairs.append((personA,personB))

## Skeleton of Group Assignments

In [198]:
remaining_groupA_dict = {}
number_groupA_dict= {}
remaining_groupB_dict = {}
number_groupB_dict = {}


for i in range(1, 1 + number_sessions):
    remaining_groupA_i = groupA_scheduling_conflicts[f"meeting_{i}_conflicts"]
    number_groupA_i = len(remaining_groupA_i)

    remaining_groupA_dict[i] = remaining_groupA_i
    number_groupA_dict[i] = number_groupA_i

for i in range(1, 1 + number_sessions):
    remaining_groupB_i = groupB_scheduling_conflicts[f"meeting_{i}_conflicts"]
    number_groupB_i = len(remaining_groupB_i)

    remaining_groupB_dict[i] = remaining_groupB_i
    number_groupB_dict[i] = number_groupB_i

### Group Framework for all sessions

In [199]:
group_frameworks = {}

In [200]:
more_than_4A_groups = [i for i in range(1, max_number_groups+1)]  # All groups can have more than 4 at the moment:

# group_frameworks = {}

for i in main_sessions:
    # Use locals() to get the local variable namespace and access the value of the variable
    all_groups = range(1, number_groups_dict[i] + 1)
    #print(f"Session {i}: {locals()[f'number_groups_{i}']}")

    # Set up a single dictionary for this session 
    group_framework = {}

    # This is who is available for this session 
    remaining_A = number_groupA_dict[i]
    remaining_B = number_groupB_dict[i]
    
    #empty list for each group 
    for group in all_groups:
        group_contents = []
        group_framework['Group ' + str(group)] = tuple(group_contents)

    # Assign group B in a round robin fashion to evenly distribute
    while remaining_B >0:
        for group in all_groups:
            if remaining_B > 0:
                group_contents = list(group_framework[f'Group {group}'])
                group_contents.append('B')
                remaining_B -= 1
                group_framework['Group ' + str(group)] = tuple(group_contents)
            else: 
                break

    # Assign group A in a round robin fashion to evenly distribute
    while remaining_A >0:
        for group in all_groups:
            assignments = group_framework[f'Group {group}']
            # if there is an open seat at the tabe:
            if len(assignments)< max_group_size:
                # Only proceed if there is an open seat at the group (<8) and dont seat where there is a 4:2 ratio
                # if not (assignments.count('A') == 4 and assignments.count('F') == 2) :
                if remaining_A > 0:
                        group_contents = list(group_framework[f'Group {group}'])
                        group_contents.append('A')
                        remaining_A -= 1
                        group_framework['Group ' + str(group)] = tuple(group_contents)
                else: 
                    break
    # Check if all group B members have been assigned
    if remaining_B > 0:
        print(f'Session {i}: Not all of groupB assigned due to group constraints! {remaining_B} of group B remaining.')
    if remaining_A > 0:
        print(f'Session {i}: Not all of groupB assigned due to group constraints! {remaining_A} of group A remaining.')


    # Sort the values of each key alphabetically
    for key in group_framework.keys():
        group_framework[key] = tuple(sorted(group_framework[key]))

    # Add the group framework for this session to the dictionary
    group_frameworks[f'Session {i}'] = group_framework

### Format & Output Dataframes

In [201]:
for i in range(1, number_sessions + 1):
    session_groups = group_frameworks[f'Session {i}']
    print(f'Session {i} : All groups are seated?',
        len(list(session_groups.values())) == number_groups_dict[i])

Session 1 : All groups are seated? True


##### Quality Check - Check that all groups are utilized in each session

In [202]:
for i in range(1, number_sessions + 1):
    session_groups = group_frameworks[f'Session {i}']
    print(f'Session {i} : All groups are seated?',
        len(list(session_groups.values())) == number_groups_dict[i])

Session 1 : All groups are seated? True


### Format & Output Dataframes

In [203]:
## Create dataframe from dictionary
group_frameworks_df = pd.DataFrame(group_frameworks)

### Total Ratio Value Counts 

In [204]:
# Create a dictionary to store the counts for each ratio
ratio_counts = {}

# Iterate over sessions and groups

for session, groups in group_frameworks.items():
    for group, contents in groups.items():
        # Calculate the ratio of A to B for each group
        ratio = f"{contents.count('A')}/{contents.count('B')}"
        
        # Increment the count in the dictionary
        ratio_counts.setdefault(ratio, {}).setdefault(session, 0)
        ratio_counts[ratio][session] += 1

# Convert the dictionary to a DataFrame
df_counts = pd.DataFrame(ratio_counts)

# Transpose the DataFrame for the desired format
ratio_counts_df = df_counts.T.fillna(0).astype(int).sort_index()

# Rearrange to be chronological
ratio_counts_df = ratio_counts_df[[f'Session {i}' for i in range(1, number_sessions+1)]]

# print
ratio_counts_df

,Session 1
2/3,3
2/4,15
3/4,2


Add the number of groups for each Session 

In [205]:
# number_groups_df = pd.DataFrame(number_groups_dict.items(), columns=['Session', 'Number of groups']).set_index('Session').T
# number_groups_df.columns = ratio_counts_df.columns

# # Add group counts to ratio 
# ratio_counts_df = pd.concat([number_groups_df, ratio_counts_df])

# # Set secondary column index as Number of groups 
# ratio_counts_df = ratio_counts_df.T.reset_index().rename(columns = {'index':'Session'}).set_index(['Session', 'Number of groups']).T

### Create DataFrame for Total counts per Session

In [206]:
# Initialize a dictionary to store the counts
counts_dict = {}

# Iterate through sessions and groups
for session, groups in group_frameworks.items():
    session_counts = {'GroupA': 0, 'GroupB': 0}
    # print(session_counts)
    
    for group, contents in groups.items():

        # Count the occurrences of 'A' and 'B' in each group
        session_counts['GroupA'] += contents.count('A')
        session_counts['GroupB'] += contents.count('B')


    # Store the counts in the dictionary
    counts_dict[(session, 'GroupA')] = session_counts['GroupA']
    counts_dict[(session, 'GroupB')] = session_counts['GroupB']
    
data = counts_dict

In [207]:
# Initialize a dictionary to store the counts
data = {}

# Iterate through sessions and groups
for session, groups in group_frameworks.items():
    session_counts = {'GroupA': 0, 'GroupB': 0}
    
    for group, contents in groups.items():
        # Count the occurrences of 'A' and 'F' in each group
        session_counts['GroupA'] += contents.count('A')
        session_counts['GroupB'] += contents.count('B')

    # Store the counts in the dictionary
    data[(session, 'GroupA')] = session_counts['GroupA']
    data[(session, 'GroupB')] = session_counts['GroupB']


# Create a dataframe from the dictionary
AB_counts_df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Rename the row and column indices
AB_counts_df.index.names = ['Session']

# Print the dataframe
AB_counts_df = AB_counts_df.reset_index()

AB_counts_df['Session'] = AB_counts_df['Session'].apply(str)
AB_counts_df[['Session', 'Role']] = AB_counts_df['Session'].str.split(',', expand=True)
AB_counts_df['Session'] = AB_counts_df['Session'].str[2:-1].str.strip()
AB_counts_df['Role'] = AB_counts_df['Role'].str[2:-2].str.strip()
AB_counts_df = AB_counts_df.rename(columns = {'Value':'Count'})

# Set index as Session and Role
AB_counts_df = AB_counts_df.set_index(['Session', 'Role'])

# pivot the table so that Sessions are across the top and people are rows
AB_counts_df = AB_counts_df.reset_index().pivot(index='Role', columns='Session', values='Count')

# filter to only rows that have 'GroupA' or 'GroupB'
AB_counts_df = AB_counts_df[AB_counts_df.index.isin(['GroupA', 'GroupB'])]

# remove the name of the columns index
AB_counts_df.columns.name = None

# Preview df
AB_counts_df

,Session 1
Role,
GroupA,42
GroupB,77


# Create Assigments

In [208]:
main_group_assignments = {}
for session in main_sessions:
    main_group_assignments[f'Session {session}'] = {}

## Set up LP Model


### Define LP Problem

In [209]:
if gurobi_lp == True: 
    # Create a Gurobi model
    lp_problem = Model("GroupAssignmentsMainSessions")
else: 
    lp_problem = pulp.LpProblem("GroupAssignmentsMainSessions", pulp.LpMaximize)

### Define Variables

Seated 
- One variable for each person in each session as each group
- Binary: 0 (person is not seated at this group in this session) or 1 (person is seated at this group in this session)

In [210]:
if gurobi_lp == True:   
    seated_vars = {(person, session, group, 'A'): lp_problem.addVar(vtype=GRB.BINARY, name=f"Seated{person}_{session}_{group}")
                for person in groupA
                for session in groupA_available_sessions[person] #main_sessions
                for group in range(1, number_groups_dict[session]+1)}

    # Update the dictionary with group B
    seated_vars.update({(person, session, group, 'B'): lp_problem.addVar(vtype=GRB.BINARY, name=f"Seated{person}_{session}_{group}")
                        for person in groupB
                        for session in groupB_available_sessions[person] #main_sessions
                        for group in range(1, number_groups_dict[session]+1)} )
else:
    seated_vars = {(person, session, group, 'A'): pulp.LpVariable(f"Seated{person}_{session}_{group}", cat='Binary')
                for person in groupA
                for session in groupA_available_sessions[person] #main_sessions
                for group in range(1, number_groups_dict[session]+1) if session in groupA_available_sessions[person]}

    # Update the dictionary with group B
    seated_vars.update({(person, session, group, 'B'): pulp.LpVariable(f"Seated{person}_{session}_{group}", cat='Binary')
                        for person in groupB
                        for session in groupB_available_sessions[person] #main_sessions
                        for group in range(1, number_groups_dict[session]+1) })

Session group Pairs
- One variable for each pair for each session and group
- Binary: 0(this pair is not seated together in this session, at this group) or 1 (this pair is seated together at this group in this session)

In [211]:
if gurobi_lp == True:
    # Create integer decision variables for pair counts
    group_session_pair_vars = {(pair, session, group): lp_problem.addVar(vtype=GRB.BINARY, name=f"pair_session_tabe{ pair, session, group}")
        for pair in pair_count_list for session in main_sessions for group in range(1, number_groups_dict[session]+1)}
else:
    # Create integer decision variables for pair counts using PuLP
    group_session_pair_vars = {(pair, session, group): pulp.LpVariable(f"pair_session_tabe{ pair, session, group}", cat='Binary')
        for pair in pair_count_list for session in main_sessions for group in range(1, number_groups_dict[session]+1)}

### Define Constraints

Assign group Session Pair Variable values based on seated variable values 
- Value should be 1 if both seated vars for this pair, session, and group are also 1 (else 0)

In [212]:
if gurobi_lp:
    # For AA Pairs 
    for pair in [pair for pair in pair_count_list if pair[0].endswith('A') and pair[1].endswith('A')]:
        for session in main_sessions:
            if session in groupA_available_sessions[pair[0]] and session in groupA_available_sessions[pair[1]]:
                for group in range(1, number_groups_dict[session]+1):
                    lp_problem.addConstr(group_session_pair_vars[(pair[0], pair[1]), session, group] >= seated_vars[pair[0], session, group, 'A'] + seated_vars[pair[1], session, group, 'A'] - 1)
                    lp_problem.addConstr(group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[0], session, group, 'A'])
                    lp_problem.addConstr(group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[1], session, group, 'A'])
    # For BB Pairs 
    for pair in [pair for pair in pair_count_list if pair[0].endswith('B') and pair[1].endswith('B')]:
        for session in main_sessions:
            if session in groupB_available_sessions[pair[0]] and session in groupB_available_sessions[pair[1]]:
                for group in range(1, number_groups_dict[session]+1):
                    lp_problem.addConstr(group_session_pair_vars[(pair[0], pair[1]), session, group] >= seated_vars[pair[0], session, group, 'B'] + seated_vars[pair[1], session, group, 'B'] - 1)
                    lp_problem.addConstr(group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[0], session, group, 'B'])
                    lp_problem.addConstr(group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[1], session, group, 'B'])
    # For AB Pairs 
    for pair in [pair for pair in pair_count_list if pair[0].endswith('A') and pair[1].endswith('B')]:
        for session in main_sessions:
            if session in groupA_available_sessions[pair[0]] and session in groupB_available_sessions[pair[1]]:
                for group in range(1, number_groups_dict[session]+1):
                    lp_problem.addConstr(group_session_pair_vars[(pair[0], pair[1]), session, group] >= seated_vars[pair[0], session, group, 'A'] + seated_vars[pair[1], session, group, 'B'] - 1)
                    lp_problem.addConstr(group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[0], session, group, 'A'])
                    lp_problem.addConstr(group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[1], session, group, 'B'])
else:
    # For AA Pairs 
    for pair in [pair for pair in pair_count_list if pair[0].endswith('A') and pair[1].endswith('A')]:
        for session in main_sessions:
            for group in range(1, number_groups_dict[session]+1):
                lp_problem += group_session_pair_vars[(pair[0], pair[1]), session, group] >= seated_vars[pair[0], session, group, 'A'] + seated_vars[pair[1], session, group, 'A'] - 1
                lp_problem += group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[0], session, group, 'A']
                lp_problem += group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[1], session, group, 'A']
    # For BB Pairs 
    for pair in [pair for pair in pair_count_list if pair[0].endswith('B') and pair[1].endswith('B')]:
        for session in main_sessions:
            for group in range(1, number_groups_dict[session]+1):
                lp_problem += group_session_pair_vars[(pair[0], pair[1]), session, group] >= seated_vars[pair[0], session, group, 'B'] + seated_vars[pair[1], session, group, 'B'] - 1
                lp_problem += group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[0], session, group, 'B']
                lp_problem += group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[1], session, group, 'B']
    # For AB Pairs 
    for pair in [pair for pair in pair_count_list if pair[0].endswith('A') and pair[1].endswith('B')]:
        for session in main_sessions:
            for group in range(1, number_groups_dict[session]+1):
                lp_problem += group_session_pair_vars[(pair[0], pair[1]), session, group] >= seated_vars[pair[0], session, group, 'A'] + seated_vars[pair[1], session, group, 'B'] - 1
                lp_problem += group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[0], session, group, 'A']
                lp_problem += group_session_pair_vars[(pair[0], pair[1]), session, group] <= seated_vars[pair[1], session, group, 'B']

Each Attendee must be sat 1 time in each session - no more and no less - for the sessions that they are available
- This combined with the constraint of how many people per session will satisfy the constraint of scheduling conflicts
- If these are all seated there is no room for people that are not available

In [213]:
if gurobi_lp == True: 
    # Constraints for each A being seated once in a session
    for personA in groupA:
        for session in list(set(groupA_available_sessions[personA]).intersection(set(main_sessions))):
            lp_problem.addConstr(grb.quicksum(seated_vars[(personA, session, group, 'A')] for group in range(1, number_groups_dict[session] + 1)) == 1)
    for personB in groupB:
        for session in list(set(groupB_available_sessions[personB]).intersection(set(main_sessions))):
            lp_problem.addConstr(grb.quicksum(seated_vars[(personB, session, group, 'B')] for group in range(1, number_groups_dict[session] + 1)) == 1)

else:
    # Constraints for each A being seated once in a session using PuLP
    for personA in groupA:
        for session in list(set(groupA_available_sessions[personA]).intersection(set(main_sessions))):
            lp_problem += pulp.lpSum(seated_vars[(personA, session, group, 'A')] for group in range(1, number_groups_dict[session] + 1)) == 1
    for personB in groupB:
        for session in list(set(groupB_available_sessions[personB]).intersection(set(main_sessions))):
            lp_problem += pulp.lpSum(seated_vars[(personB, session, group, 'B')] for group in range(1, number_groups_dict[session] + 1)) == 1

No pair can sit together more than N time in the main sessions

In [214]:
if gurobi_lp:
    for pair in pair_count_list:
        lp_problem.addConstr(grb.quicksum([group_session_pair_vars[var] for var in group_session_pair_vars if var[0] == pair]) <= 3)
else:
    for pair in pair_count_list:
        lp_problem += pulp.lpSum([group_session_pair_vars[var] for var in group_session_pair_vars if var[0] == pair]) <= 3, f"UpperBoundConstraint_{pair}"

Number of Group A members and Group B members at each group need to follow the previously identified framework

In [215]:
if gurobi_lp == True: 
    # Constraints for A and B at each group in each session
    for session in main_sessions:
        for group in range(1, number_groups_dict[session] + 1):
            assignments = group_frameworks[f'Session {session}'][f'Group {group}']
            count_A = assignments.count('A')
            count_B = assignments.count('B')

            # Constraints for A at this group in this session
            lp_problem.addConstr(grb.quicksum(seated_vars[(person, session, group, 'A')] for person in groupA_scheduling_conflicts[f'meeting_{session}_conflicts'])== count_A)

            # Constraints for B at this group in this session
            lp_problem.addConstr(grb.quicksum(seated_vars[(person, session, group, 'B')] for person in groupB_scheduling_conflicts[f'meeting_{session}_conflicts'])== count_B)

else:
    # Constraints for A and B at each group in each session using PuLP
    for session in main_sessions:
        for group in range(1, number_groups_dict[session] + 1):
            assignments = group_frameworks[f'Session {session}'][f'Group {group}']
            count_A = assignments.count('A')
            count_B = assignments.count('B')

            # Constraints for A at this group in this session
            lp_problem += pulp.lpSum(seated_vars[(person, session, group, 'A')] for person in groupA_scheduling_conflicts[f'meeting_{session}_conflicts']) == count_A

            # Constraints for B at this group in this session
            lp_problem += pulp.lpSum(seated_vars[(person, session, group, 'B')] for person in groupB_scheduling_conflicts[f'meeting_{session}_conflicts'])== count_B

Pairs that do not want to be seated together are not seated together

In [216]:
for pair in groupB_nopref_pairs:
    lp_problem.addConstr(grb.quicksum([group_session_pair_vars[var] for var in group_session_pair_vars if var[0] == pair]) == 0)

### Define Objective Function(s)

In [217]:
weight_max_mutual_prefs = 1
weight_max_groupB_prefs = 0.7
weight_max_groupA_prefs = .5
weight_min_groupB_no_prefs = 0.5

print('weight_max_mutual_prefs:         ', weight_max_mutual_prefs) 
print('weight_max_groupB_prefs:         ', weight_max_groupB_prefs)
print('weight_max_groupA_prefs:         ', weight_max_groupA_prefs)
print('weight_min_groupB_no_prefs:      ', weight_min_groupB_no_prefs)

weight_max_mutual_prefs:          1
weight_max_groupB_prefs:          0.7
weight_max_groupA_prefs:          0.5
weight_min_groupB_no_prefs:       0.5


In [218]:
# These objectives only get added if the weights are > 0 (or the minimization weights are <0)
if gurobi_lp: 

    # Maximize the number of joint-preferences satisfied
    if abs(weight_max_mutual_prefs) >0:
        print('Mutal Prefs goal applied')
        lp_problem.setObjective(\
            grb.quicksum(group_session_pair_vars[var] for var in group_session_pair_vars if var[0] in mutual_pref_pairs),
            sense=GRB.MAXIMIZE)
    
    # Maximize the number of groupB preferences satisfied
    if abs(weight_max_groupB_prefs) >0:
        print('GroupB Prefs goal applied')
        lp_problem.setObjectiveN(
            grb.quicksum(group_session_pair_vars[var] for var in group_session_pair_vars if var[0] in groupB_pref_pairs), 
            index = 5, 
            # priority = 1,
            weight=weight_max_groupB_prefs,
            name="Max GroupB Prefs" )
    
    # Maximize the number of groupA preferences satisfied
    if abs(weight_max_groupA_prefs) >0 :
        print('GroupA Prefs goal applied')
        lp_problem.setObjectiveN(
            grb.quicksum(group_session_pair_vars[var] for var in group_session_pair_vars if var[0] in groupA_pref_pairs), 
            index = 4, 
            # priority = 2,
            weight = weight_max_groupA_prefs,
            name="Max GroupA Prefs" )

    # Minimize the number  of no-seated preferences
    if abs(weight_min_groupB_no_prefs) >0 :
        print('Group B no matches goal applied')
        lp_problem.setObjectiveN(
            grb.quicksum(group_session_pair_vars[var] for var in group_session_pair_vars if var[0] in groupB_nopref_pairs),
            index = 1,
            # priority = 2,
            weight = weight_min_groupB_no_prefs , 
            name = 'Min GroupB No prefs')

In [219]:
if not gurobi_lp: 

    # Maximize the number of joint-preferences satisfied
    if abs(weight_max_mutual_prefs) > 0:
        print('Mutal Prefs goal applied')
        lp_problem += pulp.lpSum(group_session_pair_vars[var] for var in group_session_pair_vars if var[0] in mutual_pref_pairs)
    
    # Maximize the number of B preferences satisfied
    if abs(weight_max_groupB_prefs) > 0:
        print('Group B Prefs goal applied')
        lp_problem += pulp.lpSum(group_session_pair_vars[var] for var in group_session_pair_vars if var[0] in groupB_pref_pairs) * weight_max_groupB_prefs
    
    # Maximize the number of A preferences satisfied
    if abs(weight_max_groupA_prefs) > 0:
        print('Group A Prefs goal applied')
        lp_problem += pulp.lpSum(group_session_pair_vars[var] for var in group_session_pair_vars if var[0] in groupA_pref_pairs) * weight_max_groupA_prefs

    # Minimize the number  of no-seated preferences
    if abs(weight_min_groupB_no_prefs) > 0:
        print('Group B no matches goal applied')
        lp_problem += pulp.lpSum(group_session_pair_vars[var] for var in group_session_pair_vars if var[0] in groupB_nopref_pairs) * weight_min_groupB_no_prefs

Mutal Prefs goal applied
Group B Prefs goal applied
Group A Prefs goal applied
Group B no matches goal applied


### Solve the model

In [220]:
if gurobi_lp == True:
    # lp_problem.setParam('TimeLimit', 600)
    lp_problem.setParam('Symmetry', 2)
    lp_problem.setParam('NormAdjust', 2)
    # lp_problem.setParam('MIPFocus', 2)
    # lp_problem.setParam('Presolve', 1)
    
    lp_problem.setParam(GRB.Param.MIPGap, 1.5)

    # Optimize the model
    lp_problem.optimize()

    # Print results
    if lp_problem.status == GRB.OPTIMAL:
        print("Optimal solution found")

    else:
        raise ValueError("Optimal solution not found")
    
else:
    lp_problem.solve() # pulp.GLPK(msg=True)) #(pulp.COIN_CMD()) 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/6f/4qxwc5z945bfn2tp69gkk7200000gn/T/005e44ce62af481d8b3e85ae2249b028-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/6f/4qxwc5z945bfn2tp69gkk7200000gn/T/005e44ce62af481d8b3e85ae2249b028-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 428445 COLUMNS
At line 1842167 RHS
At line 2270608 BOUNDS
At line 2413410 ENDATA
Problem MODEL has 428440 rows, 142801 columns and 1128120 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 1.09 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 16560 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 17482 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6127 str

Convert variable values assigned by the model to an interpretable output

In [221]:
if gurobi_lp: 
    ## Pull Values ==1 ##
    results_list = []

    # Print the results
    print('\nResults for the Meeting Optimization:\n')
    for key, variable in seated_vars.items():
        if variable.x == 1:
            results_list.append(key)
else:
    results_list = []

    # Print the results
    print('\nResults for the Meeting Optimization:\n')
    for key, variable in seated_vars.items():
        if variable.value() == 1:
            results_list.append(key)


Results for the Meeting Optimization:



In [222]:
# Create dictionary to store pair counts # 
main_pair_counts = {}
for pair in pair_count_list:
    main_pair_counts[pair] = 0
    for session in main_sessions: 
        for group in main_group_assignments[f'Session {session}']:
            assignments = main_group_assignments[f'Session {session}'][group]
            if pair[0] in assignments and pair[1] in assignments:
                main_pair_counts[pair] += 1
main_pair_counts = {k: v for k, v in main_pair_counts.items() if v>0}

In [223]:
main_satisfied_groupB_pref_pairs = list(set([pair for pair in groupB_pref_pairs if pair in main_pair_counts.keys() or (pair[1], pair[0]) in main_pair_counts.keys()]))
main_satisfied_groupA_pref_pairs =list(set([pair for pair in groupA_pref_pairs if pair in main_pair_counts.keys() or (pair[1], pair[0]) in main_pair_counts.keys()]))
main_satisfied_mutual_pref_pairs = list(set([pair for pair in mutual_pref_pairs if pair in main_pair_counts.keys()  or (pair[1], pair[0]) in main_pair_counts.keys()]))
main_satisfied_groupB_nopref_pairs = list(set([pair for pair in groupB_nopref_pairs if pair in main_pair_counts .keys() or (pair[1], pair[0]) in main_pair_counts.keys()]))

In [225]:
print('MAIN')
if  len(groupB_pref_pairs) > 0:
    print(round((len(set(main_satisfied_groupB_pref_pairs)) / len(groupB_pref_pairs) )*100, 2) , '% GroupB prefs satsified  in Main')
if  len(groupA_pref_pairs) > 0:
    print(round((len(set(main_satisfied_groupB_pref_pairs)) / len(groupA_pref_pairs))*100, 2) , '% GroupA prefs satsified in Main')
if  len(mutual_pref_pairs) > 0:
    print(round((len(set(main_satisfied_mutual_pref_pairs)) / len(mutual_pref_pairs))*100, 2) , '% Mutual prefs satsified in Main')
# if  len(groupB_no_pref_pairs) > 0:
#     print(round((len(set(main_satisfied_groupB_nopref_pairs)) / len(groupB_no_pref_pairs))*100, 2), '% No prefs satsified  in Main')

MAIN
0.0 % GroupA prefs satsified in Main


# Reporting

##### Create dictionary with all values! 

### Look at pair counts


In [226]:
pair_counts = {}

for pair in pair_count_list:
    
    pair_counts[pair] = 0 
    if pair in main_pair_counts:
        pair_counts[pair] += main_pair_counts[pair]

In [227]:
Counter(pair_counts.values())

# print(f'{round((Counter(pair_counts.values())[1]*100 / Counter(pair_counts.values())[1]),1)}% of pairs sat together 1 time')
# print(f'{round((Counter(pair_counts.values())[2]*100 / Counter(pair_counts.values())[1]),2)}% of pairs sat together 2 times')
# print(f'{round((Counter(pair_counts.values())[3]*100 / Counter(pair_counts.values())[1]),3)}% of pairs sat together 3 times')
# print(f'{round((Counter(pair_counts.values())[4]*100 / Counter(pair_counts.values())[1]),4)}% of pairs sat together 4 times')

Counter({0: 7021})

##### Quality Check for pair counts

##### Sort the pair_counts so that they match up with the pair_count_list which creates all the table_session_pair_variables

In [228]:
pair_counts_sorted = {}
for pair in pair_counts:
    # if the inverse of the pair is in pair counts list swap them 
    if pair not in pair_count_list:
        if (pair[1], pair[0]) in pair_count_list:
            pair_counts_sorted[(pair[1], pair[0])] = pair_counts[pair]
    elif pair in pair_count_list:
        pair_counts_sorted[pair] = pair_counts[pair]

pair_counts = pair_counts_sorted

# sys.exit()

##### Check that the number of Group A  and Group B are correct for each
- If youre looking at an old rerun it may be incorrect since the updated data determines the group frameworks 

this might throw errors if the lunch session was run as a main session  

In [ ]:
for session in main_sessions:

    for group in range(1, number_groups_dict[session] + 1):
        assignments = main_group_assignments[f'Session {session}'][group]
        framework = group_frameworks[f'Session {session}'][f'Group {group}']
        if framework.count('B') != sum(1 for item in assignments if item.endswith('_B')):
            print('Session', session,'Group', group, 'GroupB')
            print('Group Framework Count B:', (framework.count('B')),'Results Count B:', (sum(1 for item in assignments if item.endswith('_B'))) )
        if framework.count('A') != sum(1 for item in assignments if item.endswith('_A')):
            print('Session', session,'Group', group, 'GroupA')
            print('Group Framework Count A:', (framework.count('A')),'Results Count A:', (sum(1 for item in assignments if item.endswith('_A'))) )
        if framework.count('A') == sum(1 for item in assignments if item.endswith('_A')) and framework.count('A') == sum(1 for item in assignments if item.endswith('_A')):
            pass

##### Are people seated more than one time in a session?


In [ ]:
def convert_session_to_list_of_lists(session_data):
    result = []
    for key, inner_dict in session_data.items():
        names = [name for name in inner_dict.values() if isinstance(name, str)]
        result.append(names)
    return result

# Create a Pandas Series
for session in range(1, number_sessions+1):
    df = pd.DataFrame(pd.Series([value for sublist in main_group_assignments[f'Session {session}'].values() for value in sublist]).value_counts()).reset_index()
    df = df[df['index']!= 'Empty Seat']
    if df[df['count'] >1].shape[0] >0:
        to_print = df[df['count'] >1].shape[0]
        raise ValueError(f'There are {to_print} names in Session {session} that are seated twice')
    #     print(f'There are {(df[df['count'] >1].shape[0])} names in Session {session} that are seated twice')